# Recomendation of similar/complementary clothes

## Load libraries and trained models

In [6]:
import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
import tensorflow as tf

In [ ]:
# Load the trained model of feature recognitions
model = load_model('model.h5')

In [ ]:
# Load model ResNet50 for embedding vectors calculation
resnet_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', pooling='avg')

In [93]:
# Load the cloth items from the database with correpondent features an embedding vectors
df = pd.read_parquet('embeddings_df1.parquet')

## Load picture of your cloth

In [ ]:
# Load user image
user_image_path = 'user/imag'
user_image = cv2.imread(user_image_path)

## Recognize features from the image (Color, Type, Pattern, Index)

In [ ]:
# Resize the image to 80x80
IMG_SIZE_FOR_RECOGNITION = 80
user_image_for_recognition = cv2.resize(user_image, (IMG_SIZE_FOR_RECOGNITION, IMG_SIZE_FOR_RECOGNITION))

In [ ]:
# Resize the image to 224x224
IMG_SIZE = 224
user_image_resized = cv2.resize(user_image, (IMG_SIZE, IMG_SIZE))

# Prediction of features
color_pred, type_pred, pattern_pred, index_pred = model.predict(np.array([user_image_for_recognition]))

# Save the recognized features to the DataFrame
pred_imag = pd.DataFrame({'product_group_name': type_pred,
                          'index_group_name': index_pred
        })

## Get embedding vector of the image

In [ ]:
# Resize the image to 224x224
IMG_SIZE_FOR_EMBEDDING = 224
user_image_for_embedding = cv2.resize(user_image, (IMG_SIZE_FOR_EMBEDDING, IMG_SIZE_FOR_EMBEDDING))

In [ ]:
# Add an extra batch dimension to the image
user_image_for_embedding = np.expand_dims(user_image_for_embedding, axis=0)

# Pass the image through the ResNet50 model
features = resnet_model.predict(user_image_for_embedding)

# Extract the embedding from the output of the model
embeddings = features.squeeze()

## Recommend similar/complementary clothes

In [103]:
# Define database to look for similar items
subset_similar = df[(df['product_group_name'] == pred_img['product_group_name']) & (df['index_group_name'] == pred_img['index_group_name'])].copy()

# Define database to look for complementary items
subset_comp = df[(df['product_group_name'] != pred_img['product_group_name']) & (df['index_group_name'] == pred_img['index_group_name'])].copy()

In [ ]:
# Caluclate similarity scores between user image and images from the database
similarity_scores = []
for array in subset_similar["embedding"]:
    array = array.reshape(1, -1)
    similarity = cosine_similarity(pred_img['embedding'].reshape(1, -1), array)
    similarity_scores.append(similarity[0][0])
subset_similar['score'] = similarity_scores

In [ ]:
# Get TOP 10 similar products
reccs = subset_similar.sort_values(by='score', ascending=False)[1:11]
reccs['article_id']

In [ ]:
# !git branch
# !git add .
# !git commit -m "final notebook for recommendation update"
# !git push origin kuzma
